In [2]:
! pip install bertopic



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 3.1 MB/s 
     |████████████████████████████████| 88 kB 6.9 MB/s 
     |████████████████████████████████| 85 kB 3.2 MB/s 
     |████████████████████████████████| 5.2 MB 31.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 4.4 MB 43.3 MB/s 
     |████████████████████████████████| 1.2 MB 28.4 MB/s 
     |████████████████████████████████| 101 kB 10.1 MB/s 
     |████████████████████████████████| 636 kB 51.0 MB/s 
     |████████████████████████████████| 6.6 MB 34.7 MB/s 
     |████████████████████████████████| 1.1 MB 49.3 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2342115 sha256=c1007d6956b138ae09384e329010eb8ad3ad01b63d3fc61a4f67a0f2bcb47914
  Stored in directory: /root/.cache

In [3]:

from bertopic import BERTopic
import pandas as pd



In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
import re
import gensim
from nltk.stem import WordNetLemmatizer

punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'         # define a string of punctuation symbols

# Functions to clean tweets
def remove_links(tweet):
    """Takes a string and removes web links from it"""
    tweet = re.sub(r'http\S+', '', tweet)   # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet)  # remove bitly links
    tweet = tweet.strip('[link]')   # remove [links]
    tweet = re.sub(r'pic.twitter\S+','', tweet)
    return tweet

def remove_users(tweet):
    """Takes a string and removes retweet and @user information"""
   # tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove re-tweet
    tweet = re.sub('(@)', '', tweet)  # remove tweeted at
    return tweet

def remove_hashtags(tweet):
    """Takes a string and removes any hash tags"""
    tweet = re.sub('(#)', ' ', tweet)  # remove hash tags
    return tweet

def remove_av(tweet):
    """Takes a string and removes AUDIO/VIDEO tags or labels"""
    tweet = re.sub('VIDEO:', '', tweet)  # remove 'VIDEO:' from start of tweet
    tweet = re.sub('AUDIO:', '', tweet)  # remove 'AUDIO:' from start of tweet
    return tweet

def tokenize(tweet):
    """Returns tokenized representation of words in lemma form excluding stopwords"""
    result = []
    for token in gensim.utils.simple_preprocess(tweet):
        if token not in gensim.parsing.preprocessing.STOPWORDS \
                and len(token) > 2:  # drops words with less than 3 characters
            result.append(lemmatize(token))
    return result

def lemmatize(token):
    """Returns lemmatization of a token"""
    return WordNetLemmatizer().lemmatize(token, pos='v')

def preprocess_tweet(tweet):
    """Main master function to clean tweets, stripping noisy characters, and tokenizing use lemmatization"""
    #tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    #tweet = remove_hashtags(tweet)
    tweet = remove_av(tweet)
    tweet = tweet.lower()  # lower case
    tweet = re.sub('[' + punctuation + ']+', ' ', tweet)  # strip punctuation
    tweet = re.sub('\s+', ' ', tweet)  # remove double spacing
  # tweet = re.sub('([0-9]+)', '', tweet)  # remove numbers
    tweet_token_list = tokenize(tweet)  # apply lemmatization and tokenization
    tweet = ' '.join(tweet_token_list)
    tweet = re.sub(' amp ', '&', tweet)
    return tweet

def basic_clean(tweet):
    """Main master function to clean tweets only without tokenization or removal of stopwords"""
    # tweet = remove_users(tweet)
    # tweet = remove_hashtags(tweet)
    tweet = remove_av(tweet)
    tweet = tweet.lower()  # lower case
    tweet = re.sub('[' + punctuation + ']+', ' ', tweet)  # strip punctuation
    tweet = re.sub('\s+', ' ', tweet)  # remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet)  # remove numbers
    tweet = re.sub(' & ', '', tweet)
    return tweet

def tokenize_tweets(df):
    df['tokens'] = df.tweet.apply(preprocess_tweet)
    num_tweets = len(df)
    print('Complete. Number of Tweets that have been cleaned and tokenized : {}'.format(num_tweets))
    return df

In [6]:
df1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/df_nlp_com2.csv",  lineterminator='\n')


In [7]:
df1_com0 = df1[df1["community"]=="community_0"]
df1_com1 = df1[df1["community"]=="community_1"]
df1_com2 = df1[df1["community"]=="community_2"]
df1_com3 = df1[df1["community"]=="community_3"]
df1_com4 = df1[df1["community"]=="community_4"]


In [8]:
joinDf = [df1_com0, df1_com1, df1_com2, df1_com3, df1_com4]

In [9]:
df = pd.concat(joinDf)

In [10]:
df = df.reset_index(drop=True)

In [11]:
df

,id,conversation_id,created_at,date,time,timezone,user_id,name,place,tweet,...,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,community,original_community_extension
0,1329210262629527557,1329042304636686336,2020-11-18 23:49:56+00:00,2020-11-18,23:49:56,100,932315271372726272,🌊🌈🦄NoraC GLORY TO UKRAINE🇺🇦! 🦄🌈🌊,NaN,@SuzeQKnits It's going to take a LOT of work &...,...,NaN,NaN,NaN,"[{'screen_name': 'SuzeQKnits', 'name': 'SuzeQK...",NaN,NaN,NaN,NaN,community_0,292
1,1301640845167407104,1301587555301896192,2020-09-03 22:58:55+01:00,2020-09-03,22:58:55,100,932315271372726272,🌊🌈🦄NoraC GLORY TO UKRAINE🇺🇦! 🦄🌈🌊,NaN,@realDonaldTrump Well we DAMNED sure need some...,...,NaN,NaN,NaN,"[{'screen_name': 'realDonaldTrump', 'name': 'D...",NaN,NaN,NaN,NaN,community_0,292
2,1344022413986721799,1344021852293898240,2020-12-29 20:48:08+00:00,2020-12-29,20:48:08,100,932315271372726272,🌊🌈🦄NoraC GLORY TO UKRAINE🇺🇦! 🦄🌈🌊,NaN,@kennethgootz They WANT #Biden &amp; his admin...,...,NaN,NaN,NaN,"[{'screen_name': 'kennethgootz', 'name': 'Ken ...",NaN,NaN,NaN,NaN,community_0,292
3,1280202034466246661,1280202034466246656,2020-07-06 19:08:44+01:00,2020-07-06,19:08:44,100,932315271372726272,🌊🌈🦄NoraC GLORY TO UKRAINE🇺🇦! 🦄🌈🌊,NaN,This is why @realDonaldTrump has got the #FED ...,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,community_0,292
4,1338185830104506374,1338185830104506368,2020-12-13 18:15:38+00:00,2020-12-13,18:15:38,100,932315271372726272,🌊🌈🦄NoraC GLORY TO UKRAINE🇺🇦! 🦄🌈🌊,NaN,#MoscowMitch wants the #economy &amp; populati...,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,community_0,292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2600,1324853685684441089,1321256114130829312,2020-11-06 23:18:27+00:00,2020-11-06,23:18:27,100,105249901,#VoteBlue2022,NaN,@zimraniaxy @zimranjacob @KingRezizt @2ordinar...,...,NaN,NaN,NaN,"[{'screen_name': 'zimranjacob', 'name': 'Zimra...",NaN,NaN,NaN,NaN,community_4,62
2601,1324852842604175364,1323724423959109632,2020-11-06 23:15:06+00:00,2020-11-06,23:15:06,100,105249901,#VoteBlue2022,NaN,@Silenced777 @Endgame2021 @NedOpposes @ivyjofr...,...,NaN,NaN,NaN,"[{'screen_name': 'Silenced777', 'name': 'Cσɾҽყ...",NaN,NaN,NaN,NaN,community_4,62
2602,1324851858771529732,1323960894787866624,2020-11-06 23:11:12+00:00,2020-11-06,23:11:12,100,105249901,#VoteBlue2022,NaN,@IamMultiversal @BobA1959 @Michmoma @KDawn___ ...,...,NaN,NaN,NaN,"[{'screen_name': 'IamMultiversal', 'name': 'Ke...",NaN,NaN,NaN,NaN,community_4,62
2603,1279860089143332869,1279860089143332864,2020-07-05 20:29:58+01:00,2020-07-05,20:29:58,100,28300282,𝐾𝑒𝑙𝑙𝑦 𝑎𝑘𝑎 𝐾𝑒𝑙𝑙𝑎𝐵𝑒𝑙𝑙𝑎,NaN,Every time Trump’s campaign makes a comment ab...,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,community_4,62


In [12]:
df = df.drop_duplicates()

In [13]:
df["tweet"] = df.tweet.astype("str")

In [14]:
import nltk


In [15]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [16]:
df["preprocessed_tweet"] = df["tweet"].apply(preprocess_tweet)


In [17]:
df.value_counts()

Series([], dtype: int64)

In [18]:
df.community.value_counts()

community_1    1210
community_0    1198
community_3     105
community_2      62
community_4      30
Name: community, dtype: int64

In [19]:
tweets = df.preprocessed_tweet.to_list()
timestamp = df.date.to_list()

In [20]:
topic_model = BERTopic(language="english", nr_topics = 30)
topics, probs = topic_model.fit_transform(tweets)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


In [21]:
topic_model.get_topic_info()


,Topic,Count,Name
0,-1,1110,-1_biden_joebiden_trump_realdonaldtrump
1,0,170,0_cannabis_food_reagan_systemicracism
2,1,132,1_realdonaldtrump_joebiden_america_vote
3,2,114,2_covid_coronavirus_vaccine_virus
4,3,107,3_realdonaldtrump_win_biden_trump
5,4,89,4_joebiden_joebide_real_know
6,5,81,5_kamalaharris_kamala_party_joebiden
7,6,66,6_racist_black_donald_trump
8,7,65,7_china_hunter_biden_job
9,8,58,8_berniesanders_cuz_ppl_centrists


In [22]:
topic_model.visualize_barchart()


In [23]:
topics_over_time = topic_model.topics_over_time(tweets, topics, timestamp, nr_bins=20)


In [24]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)


In [25]:
 topic_model.get_representative_docs(0)


['marijuana help disable veterans pain management work better neuropathic pain opioids lyrica support vet support nation wide legalization marijuana semper paralyze usmc veteran truth biden dnc democrats vote election',
 'kamalaharris bobmorrison justsayno medicareforall greennewdeal cannabis food catch marijuana jimcrow prohibition paso texas nixon reagan cannabinoids stop cancer joebiden help create dea crimebill systemicracism need bold leadership joe',
 'mariannanbcnews joebiden berniesanders dnc nbcnews mikememoli reagan democrats justsayno medicareforall cannabis cannabinoids operate immune prohibition cannabis stop cancer nixon reagan joebiden congress create dea schedule']

### Topic Modelling su tutti i tweet del secondo semestre

In [26]:
df1 = df1.drop_duplicates()

In [27]:
df1["tweet"] = df1.tweet.astype("str")

In [28]:
df1["preprocessed_tweet"] = df1["tweet"].apply(preprocess_tweet)


In [29]:
tweets = df1.preprocessed_tweet.to_list()
timestamp = df1.date.to_list()

In [30]:
topic_model = BERTopic(language="english", nr_topics = 30)
topics, probs = topic_model.fit_transform(tweets)

In [31]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,17737,-1_biden_joebiden_trump_vote
1,0,1208,0_covid_coronavirus_trump_vaccine
2,1,995,1_kamalaharris_kamala_joebiden_joebidenkamalah...
3,2,779,2_joebiden_joebide_know_hide
4,3,556,3_realdonaldtrump_rbypoll_november_joebiden
5,4,543,4_china_yrs_old_chinese
6,5,524,5_michigan_northcarolina_minnesota_foxnews
7,6,484,6_istandwithfauci_resist_cheetomcasshat_donnie...
8,7,410,7_cnn_msnbc_nbcnews_abc
9,8,407,8_peaceful_antifa_violence_blm


In [32]:
topic_model.visualize_barchart()

In [33]:
topics_over_time = topic_model.topics_over_time(tweets, topics, timestamp, nr_bins=20)

In [34]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)


### Topic modelling sulle comunità con più tweets


In [69]:
df1['community'].value_counts()

community_1       1210
community_0       1198
community_878      764
community_5        490
community_29       469
                  ... 
community_1058       1
community_1410       1
community_928        1
community_817        1
community_629        1
Name: community, Length: 1602, dtype: int64

In [70]:
df1_com0 = df1[df1["community"]=="community_0"]
df1_com1 = df1[df1["community"]=="community_1"]
df1_com5 = df1[df1["community"]=="community_5"]
df1_com878 = df1[df1["community"]=="community_878"]
df1_com29 = df1[df1["community"]=="community_29"]



In [71]:
joinDf = [df1_com0, df1_com1, df1_com5, df1_com878, df1_com29]

In [72]:
df = pd.concat(joinDf)

In [73]:
df = df.reset_index(drop=True)

In [74]:
df = df.drop_duplicates()

In [75]:
df["tweet"] = df.tweet.astype("str")

In [76]:
df["preprocessed_tweet"] = df["tweet"].apply(preprocess_tweet)

In [77]:
df.community.value_counts()

community_1      1210
community_0      1198
community_878     764
community_5       490
community_29      469
Name: community, dtype: int64

In [78]:
tweets = df.preprocessed_tweet.to_list()
timestamp = df.date.to_list()



In [79]:
topic_model = BERTopic(language="english", nr_topics = 30)
topics, probs = topic_model.fit_transform(tweets)

In [80]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,1990,-1_biden_joebiden_trump_vote
1,0,174,0_cannabis_food_reagan_medicareforall
2,1,144,1_msnbc_cnn_nbcnews_cbsnews
3,2,130,2_realdonaldtrump_joebiden_america_vote
4,3,128,3_joebiden_joebide_real_love
5,4,127,4_covid_virus_vaccine_coronavirus
6,5,123,5_realdonaldtrump_win_biden_lose
7,6,122,6_kamalaharris_kamala_joebiden_harris
8,7,112,7_tax_increase_income_raise
9,8,106,8_black_racist_donald_racial


In [81]:
topic_model.visualize_barchart()

In [82]:
topics_over_time = topic_model.topics_over_time(tweets, topics, timestamp, nr_bins=20)

In [83]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)